In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np

import pymc3 as pm
from pymc3 import  *

In [ ]:
az.style.use('arviz-darkgrid')

In [ ]:
# Value (in chilean pesos) of the permit that traders pay to fishers per legal unit
visa = 3000    

# Prior for landings (total units landed each year, considering legal and illegal)
quota = 3200000. / 27.  # 3200 ton quota in 2018. Divided in 27 because "units" are 27 kg boxes

# Other fix parameters needed for the model
wbox = 5000    # Illegal price of reference at the port per unit: This comes from my fieldwork 1
pbox = 17000   # Illegal price of reference at market per unit: This comes from my fieldwork 2
fb = 9.2e+05   # Fine expected per box, from chilean law

daysop = 200
ppmax = 6000   # Price premium higher limit (unrealistically high)
DR = 5e-05     # This comes from enforcement records data, probability of detection per box
Dmax = 3 * DR  # Detectability higher limit

# TODO shouldn't nmin be 3*quota?
nmin = quota / daysop  # Landings lower limit: 3X the legal quota is from one of our papers

nmax = (27000000. / 27.) / daysop  # Landings higher limit: 27,4K ton is from our paper as well
wl = wbox + visa  # Here, I add elasticity of demand at the port for legal, based on previous and current catch
wi = wbox         # Here, I add elasticity of demand at the port for illegal, based on previous and current catch
Pi = pbox         # Same but at the market based on illegal captures previous year

sigma = quota * (10. / 100.) / 3.  # solves 3 * sigma = 10% quota since with P(|x-mu|<3*sigma)=99.73

In [ ]:
with Model() as model: # model specifications in PyMC3 are wrapped in a with-statement
    # Define priors
    pp = Uniform('pp', 0., ppmax)
    D = Uniform('D', 0., Dmax)  # would gamma prior be better?
    nR = Uniform('nR', nmin, nmax)
    
    # Model for one time period
    pl = pbox + pp  # same but at the market based on legal captures previous year
    # Function 6 in draft, calculates the optimal quantity of illegal units. Divided by 100 to convert to ratio
    x=((((Pi - wi - pl + wl - (fb * D)) / (8 * D * pl)))) / 100.  # Illegal units
    l = 1 - x;  # legal units
    # Calculates the total (for a year)
    totallegal = (nR * l) * daysop
    
    # if we want to be able to plot posterior distributions over these intermediate variables we have to do this
    pl = pm.Deterministic('pl', pl)
    x = pm.Deterministic('x', x)
    totallegal = pm.Deterministic('totallegal', totallegal)

    # Define likelihood
    likelihood = Normal('quota', mu=totallegal, sigma=sigma, 
                        observed=quota)

    # Inference!
    trace = sample(draws=11000, tune=2000, chains=4, cores=4, target_accept=0.95) # draw 11000 posterior samples using NUTS sampling

In [ ]:
plt.figure(figsize=(7, 7))
traceplot(trace)
plt.tight_layout();